In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
brain-tumor-mri-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
import zipfile
zip_ref=zipfile.ZipFile('/content/brain-tumor-mri-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [10]:
import os
import numpy as np
import random
from PIL import Image, ImageEnhance
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.utils import shuffle

In [11]:
train_dir = '/content/Training/'
test_dir = '/content/Testing/'
train_paths = []
train_labels = []
for label in os.listdir(train_dir):
    for image in os.listdir(os.path.join(train_dir, label)):
        train_paths.append(os.path.join(train_dir, label, image))
        train_labels.append(label)

train_paths, train_labels = shuffle(train_paths, train_labels)
test_paths = []
test_labels = []
for label in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir, label)):
        test_paths.append(os.path.join(test_dir, label, image))
        test_labels.append(label)

test_paths, test_labels = shuffle(test_paths, test_labels)

In [12]:
def augment_image(image):
  image=Image.fromarray(np.uint8(image))
  image = ImageEnhance.Brightness(image).enhance(random.uniform(0.8, 1.2))
  image = ImageEnhance.Contrast(image).enhance(random.uniform(0.8, 1.2))
  image=np.array(image)/255.0
  return image
def open_images(paths):
  images=[]
  for path in paths:
    image=load_img(path,target_size=(128,128))
    image=augment_image(image)
    images.append(image)
  return np.array(images)
def encode_label(labels):
  unique_labels=os.listdir(train_dir)
  encoded=[unique_labels.index(label) for label in labels]
  return np.array(encoded)
def datagen(paths,labels,batch_size=12,epochs=1):
  for _ in range(epochs):
    for i in range(0,len(paths),batch_size):
            batch_paths = paths[i:i + batch_size]
            batch_images = open_images(batch_paths)
            batch_labels = labels[i:i + batch_size]
            batch_labels = encode_label(batch_labels)
            yield batch_images, batch_labels

In [13]:

base_model=VGG16(input_shape=(128,128,3), include_top=False, weights='imagenet')
for layer in base_model.layers:
  layer.trainable=False
base_model.layers[-2].trainable=True
base_model.layers[-3].trainable=True
base_model.layers[-4].trainable=True
model=Sequential()
model.add(Input(shape=(128,128,3)))
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
batch_size=20
steps=int(len(train_paths)/batch_size)
epochs=5
history=model.fit(datagen(train_paths,train_labels,batch_size=batch_size,epochs=epochs),epochs=epochs,steps_per_epoch=steps)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
285/285 ━━━━━━━━━━━━━━━━━━━━ 27s 60ms/step - loss: 0.6420 - sparse_categorical_accuracy: 0.7374
Epoch 2/5
285/285 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - loss: 0.2306 - sparse_categorical_accuracy: 0.9123
Epoch 3/5
285/285 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - loss: 0.1671 - sparse_categorical_accuracy: 0.9378
Epoch 4/5
285/285 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - loss: 0.1191 - sparse_categorical_accuracy: 0.9545
Epoch 5/5
285/285 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - loss: 0.0948 - sparse_categorical_accuracy: 0.9701


In [14]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import label_binarize
from tensorflow.keras.models import load_model
import numpy as np
test_images = open_images(test_paths)
test_labels_encoded = encode_label(test_labels)
test_predictions = model.predict(test_images)
print("Classification Report:")
print(classification_report(test_labels_encoded, np.argmax(test_predictions, axis=1)))

41/41 ━━━━━━━━━━━━━━━━━━━━ 13s 190ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       300
           1       0.98      1.00      0.99       405
           2       0.98      0.85      0.91       300
           3       0.87      0.97      0.92       306

    accuracy                           0.95      1311
   macro avg       0.95      0.95      0.95      1311
weighted avg       0.95      0.95      0.95      1311



In [15]:
model.save('model.h5')